In [131]:
import numpy as np
from copy import copy, deepcopy

obstaculo = "#"
cur = "X"
objetivo = "O"
up_symbol = "↑"
down_symbol = '↓'
left_symbol = '←'
right_symbol = '→'

class Maze:
    def __init__(self, lines:int, columns:int, obstacle=None, array=None) -> None:
        if not array is None: #apenas para o print_sequence
            self.maze = array
            self.lines, self.columns = self.maze.shape
        else:
            self.maze = np.empty((lines, columns), dtype= 'object') 
            self.lines, self.columns = (lines, columns)
            #ATUAL
            self.cur_line = self.lines - 1 
            self.cur_col = 0 
            self.maze[self.cur_line, self.cur_col] = cur
            
            #TARGET
            self.target_line = 0
            self.target_col = self.columns - 1
            self.maze[self.target_line, self.target_col] = objetivo
            
        self.last_move = ['last', float('inf')]
        self.cur_move = ['cur', float('-inf')]
        
        #se obstacle != None gera-se obstaculos
        if isinstance(obstacle, list):
            self.generate_obstacles(obstacle)
            
        self.move_history = [copy(self.maze)]
        
    def __str__(self) -> str:
        string = '+' + '-'*(2*self.columns - 1) + '+ \n'
        for line in self.maze:
            string += '|'
            for cell in line:
                if cell is None:
                    string += ' '
                else:
                    string += cell
                string += '|'
            string += '\n'
            string += '+' + '-'*(2*self.columns - 1) + '+ \n'
        return string
    
    def __eq__(self, other:object) -> bool:
        return np.array_equal(self.maze, other) 
    
    def __hash__(self):
        #permite usar o estado do labirinto em um conjunto (set)
        return hash(str([item for sublist in self.maze for item in sublist]))
    
    def copy(self):
        return deepcopy(self)

    def generate_obstacles(self, obstacles:list) -> None:
        for line, col in obstacles:
            self.maze[line, col] = obstaculo

    def children(self) -> list:
        functions = [self.up, self.down, self.left, self.right]
        children = []
        for function in functions:
            child = function()
            if child:
                children.append(child)
        return children
    
    # Funções de movimento
    def up(self):
        copied = self.copy()
        
        if (copied.cur_line == 0 or #na barreira superior
            copied.maze[copied.cur_line - 1, copied.cur_col] not in {None, objetivo} or #se a casa acima nao for casa válida
            (copied.maze[copied.cur_line - 1, copied.cur_col] == objetivo and np.count_nonzero(copied.maze == None) > 0) or #se tentar chegar ao target mesmo tendo casas nao visitadas
            (copied.cur_move[0] != 'up' and copied.cur_move[1] == copied.last_move[1])): #se a jogada anterior tiver o mesmo comprimento
            return None
        
        else:
            copied.maze[copied.cur_line, copied.cur_col] = up_symbol
            copied.maze[copied.cur_line - 1, copied.cur_col] = cur
            copied.cur_line -= 1
            
            if copied.cur_move[0] == 'up': copied.cur_move[1] += 1
            else: 
                copied.last_move = copied.cur_move
                copied.cur_move = ['up', 1]
                
            copied.move_history.append(copy(copied.maze))
        return copied
    
    def down(self):
        copied = self.copy()
        if (copied.cur_line == copied.lines - 1 or #na barreira inferior
            copied.maze[copied.cur_line + 1, copied.cur_col] not in {None, objetivo} or #se a casa abaixo nao for casa válida
            (copied.maze[copied.cur_line + 1, copied.cur_col] == objetivo and np.count_nonzero(copied.maze == None) > 0) or #se tentar chegar ao target mesmo tendo casas nao visitadas
            (copied.cur_move[0] != 'down' and copied.cur_move[1] == copied.last_move[1])): #se a jogada anterior tiver o mesmo comprimento
            return None
        else:
            copied.maze[copied.cur_line, copied.cur_col] = down_symbol
            copied.maze[copied.cur_line + 1, copied.cur_col] = cur
            copied.cur_line += 1
            
            if copied.cur_move[0] == 'down': copied.cur_move[1] += 1
            else: 
                copied.last_move = copied.cur_move
                copied.cur_move = ['down', 1]
                
            copied.move_history.append(copy(copied.maze))
        return copied
    
    def left(self):
        copied = self.copy()
        if (copied.cur_col == 0 or #na barreira lateral
            copied.maze[copied.cur_line, copied.cur_col - 1] not in {None, objetivo} or #se a casa esquerda nao for casa válida
            (copied.maze[copied.cur_line, copied.cur_col - 1] == objetivo and np.count_nonzero(copied.maze == None) > 0) or #se tentar chegar ao target mesmo tendo casas nao visitadas
            (copied.cur_move[0] != 'left' and copied.cur_move[1] == copied.last_move[1])): #se a jogada anterior tiver o mesmo comprimento
            return None
        else:
            copied.maze[copied.cur_line, copied.cur_col] = left_symbol
            copied.maze[copied.cur_line, copied.cur_col - 1] = cur
            copied.cur_col -= 1
            
            if copied.cur_move[0] == 'left': copied.cur_move[1] += 1
            else: 
                copied.last_move = copied.cur_move
                copied.cur_move = ['left', 1]
                
            copied.move_history.append(copy(copied.maze))
        return copied
    
    def right(self):
        copied = self.copy()
        if (copied.cur_col == copied.columns - 1 or #na barreira lateral
            copied.maze[copied.cur_line, copied.cur_col + 1] not in {None, objetivo} or #se a casa direita nao for casa válida
            (copied.maze[copied.cur_line, copied.cur_col + 1] == objetivo and np.count_nonzero(copied.maze == None) > 0) or #se tentar chegar ao target mesmo tendo casas nao visitadas
            (copied.cur_move[0] != 'right' and copied.cur_move[1] == copied.last_move[1])): #se a jogada anterior tiver o mesmo comprimento
            return None
        else:
            copied.maze[copied.cur_line, copied.cur_col] = right_symbol
            copied.maze[copied.cur_line, copied.cur_col + 1] = cur
            copied.cur_col += 1
            
            if copied.cur_move[0] == 'right': copied.cur_move[1] += 1
            else: 
                copied.last_move = copied.cur_move
                copied.cur_move = ['right', 1]
                
            copied.move_history.append(copy(copied.maze))
        return copied
    
    def is_solved(self) -> bool: 
        return np.count_nonzero(self.maze == None) == 0 and self.cur_line == self.target_line and self.cur_col == self.target_col

def print_sequence(node=None) -> None:
    if node is None:
        print('There is no solution')
        
    else:
        print("Steps: ", len(node.move_history) - 1)
        # prints the sequence of states
        for maze in node.move_history:
            print(Maze(maze.shape[0], maze.shape[1], array=maze))
            
def steps(node= None):
    return  len(node.move_history) - 1

In [132]:
from math import sqrt
from Maze import Maze

def distancia_euclidiana(maze:Maze) -> float:
    return sqrt((maze.target_line - maze.cur_line) ** 2 + (maze.target_col - maze.cur_col) ** 2)

def distancia_manhattan(maze:Maze) -> int:
    return abs(maze.target_line - maze.cur_line) + abs(maze.target_col - maze.cur_col)

In [133]:
from collections import deque
import heapq, time, random
from Maze import *
from heuristics import *

easy_mazes = [
    Maze(3,3),
    Maze(6,6, obstacle= [(0,0), (0,1), (0,2)]),
    Maze(6,6, obstacle= [(3,1), (3,2), (1,5)]),
    Maze(6,6, obstacle= [(4,1), (5,1), (1,3)]),
    Maze(6,6, obstacle= [(2,2), (4,3), (5,3)]),
    Maze(6,6, obstacle= [(0,4)]),
    Maze(6,6, obstacle= [(3,1), (4,3), (4,4)])
]

medium_mazes = [
    Maze(5,8, obstacle= [(2,0), (3,0), (0,6), (1,6)]),
    Maze(5,8, obstacle= [(1,1), (2,1), (4,6), (4,7)]),
    Maze(5,8, obstacle= [(0,0), (0,1), (0,2), (4,1)]),
    Maze(5,8, obstacle= [(0,0), (0,1), (4,6), (4,7)]),
    Maze(5,8, obstacle= [(0,6), (1,6), (2,4), (3,4)]),
    Maze(5,8, obstacle= [(0,2), (1,2), (2,4), (3,4)]),
    Maze(5,8, obstacle= [(1,3), (1,4), (1,5), (3,4)]),
    Maze(5,8, obstacle= [(0,2), (0,3)])
]

hard_mazes = [
    Maze(7,7, obstacle= [(0,3), (1,1)]),
    Maze(7,7, obstacle= [(6,1), (5,3), (3,3), (3,4)]),
    Maze(7,7, obstacle= [(2,0), (2,1), (5,0), (5,1)]),
    Maze(7,7, obstacle= [(2,0), (3,0), (3,2), (4,2)]),
    Maze(7,7, obstacle= [(2,5), (3,3), (6,5), (6,6)]),
    Maze(7,7, obstacle= [(1,1), (1,2), (3,3), (4,1)]),
    Maze(7,7, obstacle= [(3,3), (4,3), (3,6), (4,6)]),
    Maze(7,7, obstacle= [(4,1), (4,2), (5,6), (6,6)]),
    Maze(7,7, obstacle= [(3,1), (4,1), (3,6), (4,6)]),
    Maze(7,7, obstacle= [(6,1), (4,2), (4,4), (3,4)]),
    Maze(7,7, obstacle= [(4,2), (5,2), (4,5), (5,5)]),
    Maze(7,7, obstacle= [(6,1), (6,2), (1,6), (2,4)])
]

#DFS
def depth_first_search(initial_maze:Maze): #return é maze, numero de passos e tempo
    stack = deque([initial_maze]) 
    visited = set()
    steps = 0
    inicio = time.time()
    while stack:
        node = stack.pop() #get the last element that came in
        steps += 1
        if node.is_solved():
            return (node, steps, time.time() - inicio)
        visited.add(node)
        children = node.children()
        for child in children:
            if child not in visited:
                stack.append(child)
    return (None, steps, time.time() - inicio)

#Limited DFS
def depth_limited_search(initial_maze:Maze, depth_limit:int): #return é maze, numero de passos e tempo
    stack = deque([(initial_maze, 0)])  
    visited = set()
    steps = 0
    inicio = time.time()
    while stack:
        node, depth = stack.pop()
        steps += 1
        if depth < depth_limit:
            if node.is_solved():
                return (node, steps, time.time() - inicio)
            visited.add(node)
            children = node.children()
            for child in children:
                if child not in visited:
                    stack.append([child, depth + 1])
        else: break
    return (None, steps, time.time() - inicio)

#Iterative Deepening search
def iterative_deepening_search(initial_maze, depth_limit): #return é maze, numero de passos e tempo
    steps = 0
    inicio = time.time()
    for i in range(depth_limit):
        result, i_steps, *_ = depth_limited_search(initial_maze, i)
        steps += i_steps
        if result:
            return (result, steps, time.time() - inicio)
    return (None, steps, time.time() - inicio)

#BFS
def breadth_first_search(initial_maze): #return é maze, numero de passos e tempo
    queue = deque([initial_maze])  
    steps = 0
    inicio = time.time()
    while queue:
        maze = queue.popleft()   #primeiro elemento da fila (por ordem de chegada - FIFO)
        steps += 1
        if maze.is_solved(): 
            return (maze, steps, time.time() - inicio)
        for child in maze.children():   # ver as children deste nó
            queue.append(child)        
    return (None, steps, time.time() - inicio)

#Greedy
def greedy_search(initial_maze:Maze, heuristica): #return é maze, numero de passos e tempo
    setattr(Maze, "__lt__", lambda self, other: heuristica(self) < heuristica(other)) # Define um método de comparação para os mazes com base na função heurística fornecida
    priority_queue = [initial_maze]
    visited = set()
    steps = 0
    inicio = time.time()
    while priority_queue:
        atual = heapq.heappop(priority_queue) 
        steps += 1
        if atual.is_solved(): 
            return (atual, steps, time.time() - inicio)
        visited.add(atual) #labirinto visitado
        children = atual.children()
        for child in children:
            if child not in visited:
                heapq.heappush(priority_queue, child)
    return (None, steps, time.time() - inicio)

def a_star_search(maze_inicial, heuristica): #return é maze, numero de passos e tempo
        return greedy_search(maze_inicial, lambda hrst: heuristica(maze_inicial) + len(maze_inicial.move_history)) #-1=estado inicial

def  weighted_a_star_search(maze_inicial, heuristica, w): #return é maze, numero de passos e tempo
        return greedy_search(maze_inicial, lambda hrst: w*heuristica(maze_inicial) + len(maze_inicial.move_history) - 1)

import random
import pandas as pd

maze_easy = random.choice(easy_mazes)
maze_medium = random.choice(medium_mazes)
maze_hard = random.choice(hard_mazes)

depth_first = depth_first_search(maze_easy)
breadth_first =  breadth_first_search(maze_easy)
depth_limited = depth_limited_search(maze_easy, 44)
iterative_deepening = iterative_deepening_search(maze_easy, 44)



In [134]:

tabela_blind = {"algoritmos": ["depth_first_search", "breadth_first", "iterative_deepening", "depth_limited"], 
                               "Passos" : [depth_first[1], breadth_first[1], iterative_deepening[1], depth_limited[1]], 
                               "tempo" :  [depth_first[2], breadth_first[2], iterative_deepening[2], depth_limited[2]] }


df = pd.DataFrame(tabela_blind)
df

,algoritmos,Passos,tempo
0,depth_first_search,136,0.151619
1,breadth_first,4425,3.389735
2,iterative_deepening,1008,0.662295
3,depth_limited,136,0.120467


In [135]:
greedy_search_euclidiana = greedy_search(maze_easy, distancia_euclidiana)
greedy_search_manhattan = greedy_search(maze_easy, distancia_manhattan)
a_star_search_euclidiana = a_star_search(maze_easy, distancia_euclidiana)
a_star_search_manhattan = a_star_search(maze_easy, distancia_manhattan)
weighted_a_star_search_euclidiana = weighted_a_star_search(maze_easy, distancia_euclidiana, 2)
weighted_a_star_search_manhattan = weighted_a_star_search(maze_easy, distancia_manhattan, 2)

algoritmos = [greedy_search_euclidiana, greedy_search_manhattan, a_star_search_euclidiana, a_star_search_manhattan, weighted_a_star_search_euclidiana, weighted_a_star_search_manhattan]

def tabelas(lista):
    algoritmos = []
    nos = []
    tempo = []
    passos = []
    for algoritmo in lista:
        a = print_sequence(algoritmo[0])
        algoritmos.append(algoritmo[0])
        nos.append(algoritmo[1])
        tempo.append(algoritmo[2])
        passos.append(a)
    return algoritmos, nos, tempo, passos

informed = tabelas(algoritmos)


Steps:  32
+-----------+ 
| | | | | |O|
+-----------+ 
| | | | | | |
+-----------+ 
| | | | | | |
+-----------+ 
| |#| | | | |
+-----------+ 
| | | |#|#| |
+-----------+ 
|X| | | | | |
+-----------+ 

+-----------+ 
| | | | | |O|
+-----------+ 
| | | | | | |
+-----------+ 
| | | | | | |
+-----------+ 
| |#| | | | |
+-----------+ 
| | | |#|#| |
+-----------+ 
|→|X| | | | |
+-----------+ 

+-----------+ 
| | | | | |O|
+-----------+ 
| | | | | | |
+-----------+ 
| | | | | | |
+-----------+ 
| |#| | | | |
+-----------+ 
| | | |#|#| |
+-----------+ 
|→|→|X| | | |
+-----------+ 

+-----------+ 
| | | | | |O|
+-----------+ 
| | | | | | |
+-----------+ 
| | | | | | |
+-----------+ 
| |#| | | | |
+-----------+ 
| | | |#|#| |
+-----------+ 
|→|→|→|X| | |
+-----------+ 

+-----------+ 
| | | | | |O|
+-----------+ 
| | | | | | |
+-----------+ 
| | | | | | |
+-----------+ 
| |#| | | | |
+-----------+ 
| | | |#|#| |
+-----------+ 
|→|→|→|→|X| |
+-----------+ 

+-----------+ 
| | | | | |O|
+---------

In [136]:
tabela_informed ={"Algoritmos": tabelas[0], 
                               "Nós Visitados" : tabelas[1], 
                               "Tempo" : tabelas[2],
                                "Passos": tabelas[3] }

info = pd.DataFrame(tabela_informed)

info

TypeError: 'function' object is not subscriptable